# Similarity of the neighbourhoods of cities of India

## Table of Contents
- Introduction
- Data Collection
- Data Cleaning
- Methodolgy
- Analysis
- Results and Discussion
- Conclusion

## Introduction

India is the seventh largest country in the world by geographical area and world’s second-most populous country with more than 1.38 crore people. According to Census of India 2011, there are 7935 cities. The number of metropolitan cities having million plus population has also increased from 35 to 53 as per 2011 census. The name of such cities can be found here. In the current modern world citizens migrate from one place to another for a better opportunity. As per the article, Bangalore’s population comprises of 42% migrants.

The world is facing an unprecedented threat from the COVID-19 pandemic. It is difficult for a citizen who wants to explore the whole new city and choose the locality in order to find a place of reside. 

The aim of the project is to compare the cities of India to find the similarities based on the venues in the neighbourhood. This helps the working class people who may have to move to new city and are looking for the comfortable locality to stay. The locality they would want to choose is dependent on the lifestyle they carry that is in turn dependent majorly on the venues in the nearby locality. For example some people prefer to stay in a neighbourhood which has restaurants, shopping malls in the near vicinity whereas some people prefer staying near to parks, recreational zones etc. In this project, I will be making an attempt to compare the similarities of cities Pune and Bangalore City, and this methodology can be extended to compare any cities of India. Both these cities are culturally diversified, vibrant and full of opportunities for a working professional.

In [1]:
# import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

from geopy.geocoders import Nominatim 
import folium
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Data Collection

Since post office is a public facility that provides mail services, including accepting of letters and parcels, providing post office boxes to every nook and corner of the country, I will be using the all india pincode dataset from the Open Government Data(OGD) Platform India to derive the neighbourhoods of the city.

https://data.gov.in/catalog/all-india-pincode-directory?filters%5Bfield_catalog_reference%5D=85840&format=json&offset=0&limit=6&sort%5Bcreated%5D=desc

In [2]:
all_india_postal_df = pd.read_csv("all_india_PO_list_without_APS_offices_ver2_lat_long.csv")
all_india_postal_df.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
0,Achalapur B.O,504273,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Rechini S.O,Mancherial H.O,NaN,NaN
1,Ada B.O,504293,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Asifabad S.O,Mancherial H.O,NaN,NaN
2,Adegaon B.O,504307,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Boath,Adilabad,TELANGANA,NaN,Echoda S.O,Adilabad H.O,NaN,NaN
3,Adilabad Collectorate S.O,504001,S.O,Non-Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226703,NaN,Adilabad H.O,NaN,NaN
4,Adilabad H.O,504001,H.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226738,NaN,NaN,NaN,NaN


In [3]:
all_india_postal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154797 entries, 0 to 154796
Data columns (total 15 columns):
officename            154797 non-null object
pincode               154797 non-null int64
officeType            154797 non-null object
Deliverystatus        154797 non-null object
divisionname          154797 non-null object
regionname            154797 non-null object
circlename            154797 non-null object
Taluk                 146488 non-null object
Districtname          154761 non-null object
statename             154761 non-null object
Telephone             23418 non-null object
Related Suboffice     125269 non-null object
Related Headoffice    153987 non-null object
longitude             142 non-null float64
latitude              142 non-null float64
dtypes: float64(2), int64(1), object(12)
memory usage: 17.7+ MB


The dataset has attributes like office name, pincode, officeType, division, region, Taluk, District, State, latitude and longitude. But as we examine, we find that the dataset is not updated with the latitudes and the longitudes for all the records. 

To solve this problem I will be use the geopy, which is a Python client for several popular geocoding web services. The geopy makes it easy to locate the coordinates of addresses, cities, countries, and landmarks across the globe using third-party geocoders and other data sources.

Since we are interested in comparing the neighbourhood of Pune and Bangalore City, we will filter out the data of other cities.

We will also use the FourSquare API to retrieve the venues within the radius of 3 km of each neighbourhood.

In [4]:
pune_blore_df = all_india_postal_df[((all_india_postal_df['Districtname']=='Bangalore') & all_india_postal_df['Taluk'].str.contains('Bangalore')) 
                                    | ((all_india_postal_df['Districtname']=='Pune') 
                                       & ((all_india_postal_df['Taluk']=='Pune City') | (all_india_postal_df['Taluk']=='Haveli') | (all_india_postal_df['Taluk']=='Pune')))]

In [5]:
print("We have {0} neighbourhoods in Bangalore and {1} neighbourhoods in Pune".format((pune_blore_df[pune_blore_df['Districtname']=='Bangalore'].shape[0]) , pune_blore_df[pune_blore_df['Districtname']=='Pune'].shape[0]))

We have 220 neighbourhoods in Bangalore and 189 neighbourhoods in Pune


In [6]:
pune_blore_df.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
52175,A F Station Yelahanka S.O,560063,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-28478031,NaN,R T Nagar H.O,NaN,NaN
52176,Agram S.O,560007,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore South,Bangalore,KARNATAKA,080-25300192,NaN,H.A.L II Stage H.O,NaN,NaN
52177,Air Force Hospital S.O,560007,S.O,Non-Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-25302699,NaN,H.A.L II Stage H.O,NaN,NaN
52178,Amruthahalli B.O,560092,B.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,NaN,Sahakaranagar P.O S.O,R T Nagar H.O,NaN,NaN
52179,Anandnagar S.O (Bangalore),560024,S.O,Non-Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-23431761,NaN,R T Nagar H.O,NaN,NaN


In [7]:
#Removing unwanted columns: officeType, Deliverystatus, divisionname, regionname, circlename, Telephone, Related Suboffice, Related Headoffice. 
#We will also drop the longitude and latitude as most of the data is NaN
pune_blore_df.drop(columns=['officeType', 'Deliverystatus', 'divisionname', 'regionname', 'circlename', 'Telephone', 'Related Suboffice', 'Related Headoffice', 'longitude', 'latitude'], inplace=True)

C:\Users\KaranThakur\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
# Define a function to Remove the officeType(S.O, H.O, B.O) from officename
def updateOfficeName(x):
    return " ".join(x.split()[0:-1])

In [9]:
pune_blore_df.officename = pune_blore_df.officename.apply(updateOfficeName)
pune_blore_df.reset_index(drop=True, inplace=True)

C:\Users\KaranThakur\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
pune_blore_df.head()

,officename,pincode,Taluk,Districtname,statename
0,A F Station Yelahanka,560063,Bangalore North,Bangalore,KARNATAKA
1,Agram,560007,Bangalore South,Bangalore,KARNATAKA
2,Air Force Hospital,560007,Bangalore North,Bangalore,KARNATAKA
3,Amruthahalli,560092,Bangalore North,Bangalore,KARNATAKA
4,Anandnagar S.O,560024,Bangalore North,Bangalore,KARNATAKA


In [11]:
# Introducing the derived variable Neighbourhood as Concatenation of officename, Districtname and statename.
# This would help in getting the precise latitude and longitude from the geopy
pune_blore_df['Neighbourhood'] = pune_blore_df['officename']+', '+pune_blore_df['Districtname']+', '+pune_blore_df['statename']

C:\Users\KaranThakur\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
pune_blore_df.head()

,officename,pincode,Taluk,Districtname,statename,Neighbourhood
0,A F Station Yelahanka,560063,Bangalore North,Bangalore,KARNATAKA,"A F Station Yelahanka, Bangalore, KARNATAKA"
1,Agram,560007,Bangalore South,Bangalore,KARNATAKA,"Agram, Bangalore, KARNATAKA"
2,Air Force Hospital,560007,Bangalore North,Bangalore,KARNATAKA,"Air Force Hospital, Bangalore, KARNATAKA"
3,Amruthahalli,560092,Bangalore North,Bangalore,KARNATAKA,"Amruthahalli, Bangalore, KARNATAKA"
4,Anandnagar S.O,560024,Bangalore North,Bangalore,KARNATAKA,"Anandnagar S.O, Bangalore, KARNATAKA"


Lets use the geolocator API to get the Latitude and Longitude of these locations

In [13]:
pune_blore_df['Latitude']  = ''
pune_blore_df['Longitude']  = ''

C:\Users\KaranThakur\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\KaranThakur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
pune_blore_df.head()

,officename,pincode,Taluk,Districtname,statename,Neighbourhood,Latitude,Longitude
0,A F Station Yelahanka,560063,Bangalore North,Bangalore,KARNATAKA,"A F Station Yelahanka, Bangalore, KARNATAKA",,
1,Agram,560007,Bangalore South,Bangalore,KARNATAKA,"Agram, Bangalore, KARNATAKA",,
2,Air Force Hospital,560007,Bangalore North,Bangalore,KARNATAKA,"Air Force Hospital, Bangalore, KARNATAKA",,
3,Amruthahalli,560092,Bangalore North,Bangalore,KARNATAKA,"Amruthahalli, Bangalore, KARNATAKA",,
4,Anandnagar S.O,560024,Bangalore North,Bangalore,KARNATAKA,"Anandnagar S.O, Bangalore, KARNATAKA",,


In [15]:
# The number of records returned
pune_blore_df.shape

(409, 8)

In [16]:
geolocator = Nominatim(user_agent="india_explorer")
for idx,row in pune_blore_df.iterrows():
    
    try:        
        location = geolocator.geocode(row['Neighbourhood'])
        pune_blore_df.loc[idx,'Latitude'] = location.latitude
        pune_blore_df.loc[idx,'Longitude'] = location.longitude
        print("{0} Coordinates : {1}, {2}".format(row['Neighbourhood'], location.latitude, location.longitude))
    except:
        print('Coordinates not found for '+row.Neighbourhood)

Coordinates not found for A F Station Yelahanka, Bangalore, KARNATAKA
Coordinates not found for Agram, Bangalore, KARNATAKA


C:\Users\KaranThakur\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Air Force Hospital, Bangalore, KARNATAKA Coordinates : 12.964026950000001, 77.62750003174085
Amruthahalli, Bangalore, KARNATAKA Coordinates : 13.0665131, 77.5966238
Coordinates not found for Anandnagar S.O, Bangalore, KARNATAKA
Arabic College, Bangalore, KARNATAKA Coordinates : 13.0300088, 77.6208657
Attur, Bangalore, KARNATAKA Coordinates : 13.0994536, 77.5680336
Austin Town, Bangalore, KARNATAKA Coordinates : 12.9612736, 77.6152935
Banaswadi, Bangalore, KARNATAKA Coordinates : 13.0141618, 77.6518539
Coordinates not found for Bangalore Bazaar, Bangalore, KARNATAKA
Bellandur, Bangalore, KARNATAKA Coordinates : 12.93577245, 77.66676103753434
Benson Town, Bangalore, KARNATAKA Coordinates : 12.997803, 77.6041749
Bhattarahalli, Bangalore, KARNATAKA Coordinates : 13.0258004, 77.714279
Coordinates not found for Bidrahalli, Bangalore, KARNATAKA
Coordinates not found for BSF Campus Yelahanka, Bangalore, KARNATAKA
Byatarayanapura, Bangalore, KARNATAKA Coordinates : 13.062073550000001, 77.596391

Electronics City, Bangalore, KARNATAKA Coordinates : 12.8468545, 77.6769267
Coordinates not found for Gaviopuram Extension, Bangalore, KARNATAKA
Girinagar S.O, Bangalore, KARNATAKA Coordinates : 12.9401439, 77.5444995
Gottigere, Bangalore, KARNATAKA Coordinates : 12.8565317, 77.5877237
Coordinates not found for Governmemnt Electric Factory, Bangalore, KARNATAKA
HSR Layout, Bangalore, KARNATAKA Coordinates : 12.9116225, 77.6388622
Hulimavu, Bangalore, KARNATAKA Coordinates : 12.8773486, 77.6028031
Hulimavu, Bangalore, KARNATAKA Coordinates : 12.8773486, 77.6028031
J P Nagar, Bangalore, KARNATAKA Coordinates : 12.909492199999999, 77.59673855731768
Jayanagar, Bangalore, KARNATAKA Coordinates : 12.9292731, 77.5824229
Jayanagar West, Bangalore, KARNATAKA Coordinates : 12.9292731, 77.5824229
Coordinates not found for Jayangar III Block, Bangalore, KARNATAKA
JP Nagar III Phase, Bangalore, KARNATAKA Coordinates : 12.9117472, 77.5918492
JP Nagar VIII phase, Bangalore, KARNATAKA Coordinates : 12

Coordinates not found for Dapodi Bazar, Pune, MAHARASHTRA
Dapodi, Pune, MAHARASHTRA Coordinates : 18.580845699999998, 73.83277484526513
Dhanori, Pune, MAHARASHTRA Coordinates : 18.5906707, 73.8913189
Dighi Camp, Pune, MAHARASHTRA Coordinates : 18.6265187, 73.8660847
Coordinates not found for Dr.B.A. Chowk, Pune, MAHARASHTRA
Coordinates not found for Dukirkline, Pune, MAHARASHTRA
East Khadki, Pune, MAHARASHTRA Coordinates : 18.5681753, 73.8507787
Coordinates not found for Ghorpuri Bazar, Pune, MAHARASHTRA
Coordinates not found for Gondhale Nagar, Pune, MAHARASHTRA
Coordinates not found for H.E. Factory, Pune, MAHARASHTRA
Hadapsar, Pune, MAHARASHTRA Coordinates : 18.526967, 73.9278252
Coordinates not found for Hadpsar I.E., Pune, MAHARASHTRA
Coordinates not found for Iaf Station, Pune, MAHARASHTRA
Coordinates not found for Indrayaninagar, Pune, MAHARASHTRA
Coordinates not found for Infotech Park (Hinjawadi), Pune, MAHARASHTRA
Kalewadi, Pune, MAHARASHTRA Coordinates : 18.4043584, 74.02144

Ashtapur, Pune, MAHARASHTRA Coordinates : 18.5464622, 74.1315318
Coordinates not found for Cod Dehu Road, Pune, MAHARASHTRA
Coordinates not found for Dahitane, Pune, MAHARASHTRA
Coordinates not found for Dehu Road Cantt, Pune, MAHARASHTRA
Dehu, Pune, MAHARASHTRA Coordinates : 18.680047, 73.7343306
Donje, Pune, MAHARASHTRA Coordinates : 18.4121837, 73.7665893
Coordinates not found for Fulgaon, Pune, MAHARASHTRA
Coordinates not found for Gogalwadi(E), Pune, MAHARASHTRA
I.A.T, Pune, MAHARASHTRA Coordinates : 18.46239465, 73.88969221080768
Coordinates not found for Kadam Wak Wasti, Pune, MAHARASHTRA
Kalyan, Pune, MAHARASHTRA Coordinates : 18.3500438, 73.7647832
Coordinates not found for Kamdhenunagar, Pune, MAHARASHTRA
Kesnand, Pune, MAHARASHTRA Coordinates : 18.5807136, 73.9875865
Khanapur Haveli, Pune, MAHARASHTRA Coordinates : 18.4006615, 73.7318306
Khed Shivapur, Pune, MAHARASHTRA Coordinates : 18.3391133, 73.8392736
Coordinates not found for Khed Shivapur Baug, Pune, MAHARASHTRA
Kiwal

We see some duplicate entries in the dataset.

In [17]:
pune_blore_df.shape

(409, 8)

In [18]:
# Dropping the duplicate entries from the dataset.
pune_blore_df.drop_duplicates(subset ="officename", keep = False, inplace = True)

C:\Users\KaranThakur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
pune_blore_df.shape

(401, 8)

In [44]:
pune_blore_df.head()

,officename,pincode,Taluk,Districtname,statename,Neighbourhood,Latitude,Longitude
0,Air Force Hospital,560007,Bangalore North,Bangalore,KARNATAKA,"Air Force Hospital, Bangalore, KARNATAKA",12.964,77.6275
1,Amruthahalli,560092,Bangalore North,Bangalore,KARNATAKA,"Amruthahalli, Bangalore, KARNATAKA",13.0665,77.5966
2,Arabic College,560045,Bangalore North,Bangalore,KARNATAKA,"Arabic College, Bangalore, KARNATAKA",13.03,77.6209
3,Attur,560064,Bangalore North,Bangalore,KARNATAKA,"Attur, Bangalore, KARNATAKA",13.0995,77.568
4,Austin Town,560047,Bangalore North,Bangalore,KARNATAKA,"Austin Town, Bangalore, KARNATAKA",12.9613,77.6153


In [21]:
# The records for which we don't have the coordinates:
pune_blore_df[pune_blore_df.Latitude!=''].shape

(252, 8)

In [22]:
# Drop the records for which we don't have the coordinates
pune_blore_df = pune_blore_df[pune_blore_df.Latitude!='']
pune_blore_df.reset_index(drop=True, inplace=True)

Visualise the neighborhoods of Bangalore and Pune on the map.

In [23]:
PUNE_LATITUDE = geolocator.geocode('PUNE').latitude
PUNE_LONGITUDE = geolocator.geocode('PUNE').longitude
print('The geograpical coordinates of Pune are {}, {}.'.format(PUNE_LATITUDE, PUNE_LONGITUDE))

The geograpical coordinates of Pune are 18.521428, 73.8544541.


In [24]:
BANGALORE_LATITUDE = geolocator.geocode('BANGALORE').latitude
BANGALORE_LONGITUDE = geolocator.geocode('BANGALORE').longitude
print('The geograpical coordinates of Bangalore are {}, {}.'.format(BANGALORE_LATITUDE, BANGALORE_LONGITUDE))

The geograpical coordinates of Bangalore are 12.9791198, 77.5912997.


In [25]:
pune_df = pune_blore_df[pune_blore_df['Districtname'] == 'Pune']
bangalore_df = pune_blore_df[pune_blore_df['Districtname'] == 'Bangalore']

In [26]:
print('We have {0} and {1} neighbourhoods along with geographical coordinates for Pune and Bangalore respectively.'.format(pune_df.shape[0], bangalore_df.shape[0]))

We have 105 and 147 neighbourhoods along with geographical coordinates for Pune and Bangalore respectively.


In [104]:
import folium
pune_map = folium.Map(location=[PUNE_LATITUDE, PUNE_LONGITUDE], zoom_start=12)
# add markers to map
for lat, lng, label in zip(pune_df['Latitude'], pune_df['Longitude'], pune_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(pune_map)
pune_map

In [103]:
bangalore_map = folium.Map(location=[BANGALORE_LATITUDE, BANGALORE_LONGITUDE], zoom_start=12)
for lat, lng, label in zip(bangalore_df['Latitude'], bangalore_df['Longitude'], bangalore_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bangalore_map)
bangalore_map

### Foursquare API

In [29]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'YKBMLD2MG4VKNU3WTETTSR1RUMIPPEIC42OJ10W0HK2Y1CHS' # your Foursquare ID
CLIENT_SECRET = 'PK3OR2LHB0VNVOCFNBS1P3YYFOJJTITWQAINUHROPPU0VHPN' # your Foursquare Secret
VERSION = '20201109' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: YKBMLD2MG4VKNU3WTETTSR1RUMIPPEIC42OJ10W0HK2Y1CHS
CLIENT_SECRET:PK3OR2LHB0VNVOCFNBS1P3YYFOJJTITWQAINUHROPPU0VHPN


Let's explore the first neighbourhood in our dataframe.

In [30]:
pune_blore_df.loc[0, 'Neighbourhood']

'Air Force Hospital, Bangalore, KARNATAKA'

In [31]:
neighbourhood_latitude = pune_blore_df.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = pune_blore_df.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = pune_blore_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Air Force Hospital, Bangalore, KARNATAKA are 12.964026950000001, 77.62750003174085.


Now, let's get the top 100 venues that are in Air Force Hospital, Bangalore, KARNATAKA within a radius of 1000 meters.

In [32]:
# type your answer here
search_query = pune_blore_df.loc[0, 'Neighbourhood']
radius = 1000
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=YKBMLD2MG4VKNU3WTETTSR1RUMIPPEIC42OJ10W0HK2Y1CHS&client_secret=PK3OR2LHB0VNVOCFNBS1P3YYFOJJTITWQAINUHROPPU0VHPN&ll=12.964026950000001,77.62750003174085&v=20201109&radius=1000&limit=100'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc4835092d28855b9102ee9'},
 'response': {'headerLocation': 'Jogupalya',
  'headerFullLocation': 'Jogupalya, Bangalore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 12.97302695900001,
    'lng': 77.6367181975279},
   'sw': {'lat': 12.955026940999993, 'lng': 77.6182818659538}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c62b2dffa7bc928f5ab0e27',
       'name': 'Retro Lounge Bar',
       'location': {'address': '18/1, Cambridge Rd,',
        'crossStreet': 'Ulsoor',
        'lat': 12.972614445679424,
        'lng': 77.6267001093147,
        'labeledLatLngs': [{'label': 'display',
          'lat': 12.972614445679424,
          'lng': 77.6267001093147}],
        'distance'

Let's define the **get_category_type** function to get the category type

In [34]:
# Defining a function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Retro Lounge Bar,Bar,12.972614,77.626700
1,Domino's Pizza,Pizza Place,12.969042,77.629077
2,Lassi Point,Juice Bar,12.970021,77.631221
3,Chai Point,Café,12.970279,77.632180
4,"Cafe Coffee Day,",Café,12.965509,77.618498


In [36]:
print('{0} venues were returned by Foursquare in the vicinity of {1}.'.format(nearby_venues.shape[0], pune_blore_df.loc[0, 'Neighbourhood']))

11 venues were returned by Foursquare in the vicinity of Air Force Hospital, Bangalore, KARNATAKA.


In [37]:
# Function to repeat the same process to all the neighbourhoods in Pune and Bangalore
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Code to run the above function on each neighbourhood and create a new dataframe called _pune_bangalore_venues_.


In [39]:
punebangalore_venues = getNearbyVenues(pune_blore_df.Neighbourhood, pune_blore_df.Latitude, pune_blore_df.Longitude)
punebangalore_venues.head()

Air Force Hospital, Bangalore, KARNATAKA
Amruthahalli, Bangalore, KARNATAKA
Arabic College, Bangalore, KARNATAKA
Attur, Bangalore, KARNATAKA
Austin Town, Bangalore, KARNATAKA
Banaswadi, Bangalore, KARNATAKA
Bellandur, Bangalore, KARNATAKA
Benson Town, Bangalore, KARNATAKA
Bhattarahalli, Bangalore, KARNATAKA
Byatarayanapura, Bangalore, KARNATAKA
C.V.Raman Nagar, Bangalore, KARNATAKA
CMP Centre And School, Bangalore, KARNATAKA
CRPF Campus Yelahanka, Bangalore, KARNATAKA
Devasandra, Bangalore, KARNATAKA
Doddanekkundi, Bangalore, KARNATAKA
Domlur, Bangalore, KARNATAKA
Dr. Ambedkar Veedhi, Bangalore, KARNATAKA
EPIP, Bangalore, KARNATAKA
G.K.V.K., Bangalore, KARNATAKA
Gunjur, Bangalore, KARNATAKA
H.A. Farm, Bangalore, KARNATAKA
H.K.P. Road, Bangalore, KARNATAKA
Hebbal Kempapura, Bangalore, KARNATAKA
Hoodi, Bangalore, KARNATAKA
Horamavu, Bangalore, KARNATAKA
Indiranagar S.O, Bangalore, KARNATAKA
J.C.Nagar, Bangalore, KARNATAKA
Jakkur, Bangalore, KARNATAKA
Kadugodi, Bangalore, KARNATAKA
Kalyan

Phursungi, Pune, MAHARASHTRA
Pimpri Sandas, Pune, MAHARASHTRA
Ravet, Pune, MAHARASHTRA
Sangrun, Pune, MAHARASHTRA
Shivajinagar, Pune, MAHARASHTRA
Theur, Pune, MAHARASHTRA
Vadhu BK, Pune, MAHARASHTRA


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,League of Extraordinary Gamers,12.967099,77.636919,Gaming Cafe
1,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Corner House,12.969973,77.637453,Ice Cream Shop
2,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Vivanta by Taj,12.973365,77.619951,Hotel
3,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Retro Lounge Bar,12.972614,77.626700,Bar
4,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Foodhall,12.973486,77.620117,Department Store


In [40]:
print(punebangalore_venues.shape)
punebangalore_venues.head()

(9654, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,League of Extraordinary Gamers,12.967099,77.636919,Gaming Cafe
1,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Corner House,12.969973,77.637453,Ice Cream Shop
2,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Vivanta by Taj,12.973365,77.619951,Hotel
3,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Retro Lounge Bar,12.972614,77.626700,Bar
4,"Air Force Hospital, Bangalore, KARNATAKA",12.964027,77.6275,Foodhall,12.973486,77.620117,Department Store


In [41]:
punebangalore_venues[['Venue','Neighbourhood']].groupby('Neighbourhood').count()

,Venue
Neighbourhood,
"AFMC, Pune, MAHARASHTRA",29
"Adugodi, Bangalore, KARNATAKA",100
"Air Force Hospital, Bangalore, KARNATAKA",87
"Akurdi, Pune, MAHARASHTRA",26
"Ambegaon BK, Pune, MAHARASHTRA",4
"Amruthahalli, Bangalore, KARNATAKA",31
"Anjanapura, Bangalore, KARNATAKA",7
"Arabic College, Bangalore, KARNATAKA",14
"Armament, Pune, MAHARASHTRA",13


In [46]:
punebangalore_venues[['Venue','Neighbourhood']].groupby('Neighbourhood').count().describe()

,Venue
count,241.000000
mean,40.058091
std,33.645157
min,1.000000
25%,10.000000
50%,31.000000
75%,67.000000
max,100.000000


The unique categories can be curated from all the returned venues

In [47]:
print('There are {} uniques categories.'.format(len(punebangalore_venues['Venue Category'].unique())))

There are 280 uniques categories.


### Analyzing each neighborhood

In [48]:
# one hot encoding
punebangalore_onehot = pd.get_dummies(punebangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
punebangalore_onehot['Neighbourhood'] = punebangalore_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = ['Neighbourhood'] + list(punebangalore_onehot.drop(columns=['Neighbourhood']).columns)
punebangalore_onehot = punebangalore_onehot[fixed_columns]

punebangalore_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Andhra Restaurant,...,Warehouse Store,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Air Force Hospital, Bangalore, KARNATAKA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Air Force Hospital, Bangalore, KARNATAKA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Air Force Hospital, Bangalore, KARNATAKA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Air Force Hospital, Bangalore, KARNATAKA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Air Force Hospital, Bangalore, KARNATAKA",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
punebangalore_onehot.shape

(9654, 281)

Group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [50]:
punebangalore_grouped = punebangalore_onehot.groupby('Neighbourhood').mean().reset_index()
print(punebangalore_grouped.shape)
punebangalore_grouped

(241, 281)


,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Andhra Restaurant,...,Warehouse Store,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"AFMC, Pune, MAHARASHTRA",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.034483,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
1,"Adugodi, Bangalore, KARNATAKA",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
2,"Air Force Hospital, Bangalore, KARNATAKA",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.011494,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
3,"Akurdi, Pune, MAHARASHTRA",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
4,"Ambegaon BK, Pune, MAHARASHTRA",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
5,"Amruthahalli, Bangalore, KARNATAKA",0.000000,0.000000,0.000000,0.0,0.032258,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
6,"Anjanapura, Bangalore, KARNATAKA",0.285714,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
7,"Arabic College, Bangalore, KARNATAKA",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
8,"Armament, Pune, MAHARASHTRA",0.000000,0.076923,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
9,"Ashoknagar S.O, Bangalore, KARNATAKA",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


Let's print each neighborhood along with the top 5 most common venues

In [51]:
num_top_venues = 5

for hood in punebangalore_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = punebangalore_grouped[punebangalore_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AFMC, Pune, MAHARASHTRA----
               venue  freq
0  Indian Restaurant  0.10
1     Ice Cream Shop  0.07
2                Bar  0.07
3        Pizza Place  0.07
4          Multiplex  0.07


----Adugodi, Bangalore, KARNATAKA----
                venue  freq
0   Indian Restaurant  0.20
1                Café  0.08
2        Dessert Shop  0.05
3  Chinese Restaurant  0.04
4              Lounge  0.03


----Air Force Hospital, Bangalore, KARNATAKA----
                venue  freq
0   Indian Restaurant  0.13
1                Café  0.09
2               Hotel  0.08
3                 Bar  0.03
4  Chinese Restaurant  0.03


----Akurdi, Pune, MAHARASHTRA----
               venue  freq
0  Indian Restaurant  0.12
1               Café  0.12
2   Asian Restaurant  0.12
3        Snack Place  0.08
4     Ice Cream Shop  0.08


----Ambegaon BK, Pune, MAHARASHTRA----
                           venue  freq
0                          Hotel  0.25
1              Indian Restaurant  0.25
2               Asian R



----Bommasandra Industrial Estate, Bangalore, KARNATAKA----
                     venue  freq
0               Food Court   0.5
1                   Casino   0.5
2  New American Restaurant   0.0
3             Neighborhood   0.0
4              Music Venue   0.0


----Botanical Garden S.O, Pune, MAHARASHTRA----
                 venue  freq
0    Indian Restaurant  0.07
1    Electronics Store  0.07
2  Sporting Goods Shop  0.07
3          Snack Place  0.04
4                  Bar  0.04


----Byatarayanapura, Bangalore, KARNATAKA----
               venue  freq
0  Indian Restaurant  0.19
1     Ice Cream Shop  0.09
2               Café  0.09
3        Pizza Place  0.09
4   Department Store  0.06


----C D A (O), Pune, MAHARASHTRA----
                venue  freq
0         Coffee Shop  0.17
1                 Gym  0.08
2  Mughlai Restaurant  0.08
3   Indian Restaurant  0.08
4                Café  0.08


----C M E, Pune, MAHARASHTRA----
                venue  freq
0                 Gym  0.09
1  Mughl

                     venue  freq
0         Department Store   0.2
1           Ice Cream Shop   0.1
2               Restaurant   0.1
3        Indian Restaurant   0.1
4  North Indian Restaurant   0.1


----H.K.P. Road, Bangalore, KARNATAKA----
                           venue  freq
0                    Pizza Place  0.19
1  Vegetarian / Vegan Restaurant  0.06
2                            Pub  0.06
3        North Indian Restaurant  0.06
4                     Restaurant  0.06


----HSR Layout, Bangalore, KARNATAKA----
               venue  freq
0  Indian Restaurant  0.14
1               Café  0.10
2        Pizza Place  0.08
3     Ice Cream Shop  0.05
4   Department Store  0.05


----Hadapsar, Pune, MAHARASHTRA----
                venue  freq
0   Indian Restaurant  0.10
1  Italian Restaurant  0.08
2          Donut Shop  0.08
3         Coffee Shop  0.08
4  Chinese Restaurant  0.06


----Hebbal Kempapura, Bangalore, KARNATAKA----
               venue  freq
0   Department Store  0.17
1  Indian 

                  venue  freq
0                  Café  0.12
1        Ice Cream Shop  0.10
2           Coffee Shop  0.10
3      Asian Restaurant  0.06
4  Fast Food Restaurant  0.04


----Kondhwa Lh, Pune, MAHARASHTRA----
                  venue  freq
0                  Café  0.12
1        Ice Cream Shop  0.10
2           Coffee Shop  0.10
3      Asian Restaurant  0.06
4  Fast Food Restaurant  0.04


----Koramangala I Block, Bangalore, KARNATAKA----
                venue  freq
0   Indian Restaurant  0.16
1                Café  0.13
2      Ice Cream Shop  0.05
3  Italian Restaurant  0.05
4         Coffee Shop  0.04


----Kothanur, Bangalore, KARNATAKA----
                      venue  freq
0                Restaurant  0.07
1  Mediterranean Restaurant  0.07
2                    Bakery  0.07
3                      Café  0.07
4             Grocery Store  0.07


----Kothrud, Pune, MAHARASHTRA----
               venue  freq
0  Indian Restaurant  0.13
1               Café  0.08
2     Ice Cream S

               venue  freq
0  Indian Restaurant  0.09
1              Hotel  0.07
2             Lounge  0.07
3                Pub  0.05
4            Brewery  0.04


----N I B M, Pune, MAHARASHTRA----
                venue  freq
0         Coffee Shop  0.17
1                 Gym  0.08
2  Mughlai Restaurant  0.08
3   Indian Restaurant  0.08
4                Café  0.08


----N.C.L. Pune, Pune, MAHARASHTRA----
               venue  freq
0    Airport Service  0.19
1  Indian Restaurant  0.06
2        Coffee Shop  0.06
3         Smoke Shop  0.06
4          Bookstore  0.06


----N.I.A., Pune, MAHARASHTRA----
                venue  freq
0         Coffee Shop  0.17
1                 Gym  0.08
2  Mughlai Restaurant  0.08
3   Indian Restaurant  0.08
4                Café  0.08


----N.W. College, Pune, MAHARASHTRA----
               venue  freq
0  Indian Restaurant  0.15
1               Café  0.10
2              Hotel  0.07
3        Coffee Shop  0.06
4     Sandwich Place  0.05


----NAL, Bangalore, 

                venue  freq
0                Café  0.09
1              Bakery  0.08
2   Indian Restaurant  0.07
3  Seafood Restaurant  0.04
4              Lounge  0.04


----Sangrun, Pune, MAHARASHTRA----
                     venue  freq
0                   Resort   1.0
1                      ATM   0.0
2  New American Restaurant   0.0
3              Music Venue   0.0
4              Music Store   0.0


----Science Institute, Bangalore, KARNATAKA----
                  venue  freq
0     Indian Restaurant  0.23
1        Ice Cream Shop  0.15
2    Chinese Restaurant  0.10
3  Fast Food Restaurant  0.10
4                  Café  0.06


----Seshadripuram, Bangalore, KARNATAKA----
                           venue  freq
0              Indian Restaurant  0.18
1                    Coffee Shop  0.06
2                 Ice Cream Shop  0.05
3                           Café  0.05
4  Vegetarian / Vegan Restaurant  0.04


----Shaniwar Peth S.O, Pune, MAHARASHTRA----
                           venue  freq
0

                  venue  freq
0     Indian Restaurant  0.12
1        Ice Cream Shop  0.10
2  Fast Food Restaurant  0.10
3                  Café  0.10
4                Bakery  0.10


----Whitefield, Bangalore, KARNATAKA----
               venue  freq
0               Café  0.10
1  Indian Restaurant  0.10
2              Hotel  0.10
3        Pizza Place  0.06
4          Multiplex  0.06


----Wilson Garden, Bangalore, KARNATAKA----
                           venue  freq
0                           Café  0.20
1              Indian Restaurant  0.15
2             Salon / Barbershop  0.10
3                     Playground  0.05
4  Vegetarian / Vegan Restaurant  0.05


----Yelachenahalli, Bangalore, KARNATAKA----
                           venue  freq
0               Department Store  0.07
1                    Pizza Place  0.07
2                           Café  0.07
3  Vegetarian / Vegan Restaurant  0.05
4                     Food Truck  0.05


----Yelahanka, Bangalore, KARNATAKA----
            

Defining a dataframe and sort the venues in descending order.

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 most common venues for each neighbourhood.

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = punebangalore_grouped['Neighbourhood']

for ind in np.arange(punebangalore_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(punebangalore_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"AFMC, Pune, MAHARASHTRA",Indian Restaurant,Shopping Mall,Bar,Breakfast Spot,Multiplex,Ice Cream Shop,Pizza Place,Department Store,Burger Joint,Electronics Store
1,"Adugodi, Bangalore, KARNATAKA",Indian Restaurant,Café,Dessert Shop,Chinese Restaurant,Ice Cream Shop,Bookstore,Pizza Place,Lounge,Coffee Shop,Mexican Restaurant
2,"Air Force Hospital, Bangalore, KARNATAKA",Indian Restaurant,Café,Hotel,Asian Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Pizza Place,Pub,Restaurant
3,"Akurdi, Pune, MAHARASHTRA",Café,Indian Restaurant,Asian Restaurant,Snack Place,Ice Cream Shop,Convenience Store,Diner,Middle Eastern Restaurant,Fast Food Restaurant,Gaming Cafe
4,"Ambegaon BK, Pune, MAHARASHTRA",Hotel,Vegetarian / Vegan Restaurant,Asian Restaurant,Indian Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm


### Cluster Neighbourhoods

In [55]:
neighbourhoods_venues_sorted.shape

(241, 11)

In [62]:
# Choosing Number of clusters as 7 as we don't see more steepness in the slope after 7-8
kclusters = 7

punebangalore_grouped_clustering = punebangalore_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(punebangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 5, 2, 3, 2, 0, 2])

In [64]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [65]:
neighbourhoods_venues_sorted.shape

(241, 12)

In [66]:
pune_blore_df.shape

(252, 8)

In [73]:
punebangalore_merged = pune_blore_df

# merge punebangalore_grouped with punebangalore_data to add latitude/longitude for each neighborhood
punebangalore_merged = punebangalore_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

punebangalore_merged.head() # check the last columns!

,officename,pincode,Taluk,Districtname,statename,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Air Force Hospital,560007,Bangalore North,Bangalore,KARNATAKA,"Air Force Hospital, Bangalore, KARNATAKA",12.964,77.6275,2.0,Indian Restaurant,Café,Hotel,Asian Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Pizza Place,Pub,Restaurant
1,Amruthahalli,560092,Bangalore North,Bangalore,KARNATAKA,"Amruthahalli, Bangalore, KARNATAKA",13.0665,77.5966,2.0,Indian Restaurant,Ice Cream Shop,Department Store,Café,Hotel,Fast Food Restaurant,Brewery,Bubble Tea Shop,Building,Scenic Lookout
2,Arabic College,560045,Bangalore North,Bangalore,KARNATAKA,"Arabic College, Bangalore, KARNATAKA",13.03,77.6209,2.0,Pizza Place,Indian Restaurant,Convenience Store,Clothing Store,BBQ Joint,Jewelry Store,Bus Station,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tea Room
3,Attur,560064,Bangalore North,Bangalore,KARNATAKA,"Attur, Bangalore, KARNATAKA",13.0995,77.568,5.0,Indian Restaurant,Bakery,BBQ Joint,Soccer Field,Bus Station,Asian Restaurant,Andhra Restaurant,Snack Place,Deli / Bodega,Dance Studio
4,Austin Town,560047,Bangalore North,Bangalore,KARNATAKA,"Austin Town, Bangalore, KARNATAKA",12.9613,77.6153,0.0,Hotel,Indian Restaurant,Café,Clothing Store,Bakery,Pub,Chinese Restaurant,Coffee Shop,Donut Shop,Bar


In [74]:
# Check if we have any Nan
punebangalore_merged[punebangalore_merged['Cluster Labels'].isna()]

,officename,pincode,Taluk,Districtname,statename,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Agara,560034,Bangalore South,Bangalore,KARNATAKA,"Agara, Bangalore, KARNATAKA",12.6201,77.4793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,Koramangala,560034,Bangalore South,Bangalore,KARNATAKA,"Koramangala, Bangalore, KARNATAKA",13.2924,77.7519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Koramangala VI Bk,560095,Bangalore South,Bangalore,KARNATAKA,"Koramangala VI Bk, Bangalore, KARNATAKA",13.2924,77.7519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,Tavarekere,560029,Bangalore,Bangalore,KARNATAKA,"Tavarekere, Bangalore, KARNATAKA",13.1334,77.9291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,Tavarekere S.O,562130,Bangaloresouth,Bangalore,KARNATAKA,"Tavarekere S.O, Bangalore, KARNATAKA",13.1334,77.9291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,Sangavi,411027,Pune City,Pune,MAHARASHTRA,"Sangavi, Pune, MAHARASHTRA",18.0603,74.4856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,Ashtapur,412207,Haveli,Pune,MAHARASHTRA,"Ashtapur, Pune, MAHARASHTRA",18.5465,74.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,Kalyan,412205,Haveli,Pune,MAHARASHTRA,"Kalyan, Pune, MAHARASHTRA",18.35,73.7648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,Kiwale,412101,Haveli,Pune,MAHARASHTRA,"Kiwale, Pune, MAHARASHTRA",18.8497,73.8034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,Phursungi,412308,Haveli,Pune,MAHARASHTRA,"Phursungi, Pune, MAHARASHTRA",18.4573,74.0021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# We will remove the null values as we do not have venues returned by FourSquare API for all the neighborhoods.
punebangalore_merged.dropna(axis=0, inplace=True)

In [76]:
punebangalore_merged.shape

(241, 19)

In [77]:
punebangalore_merged['Cluster Labels'].value_counts()

2.0    135
0.0     67
5.0     20
3.0     10
4.0      6
1.0      2
6.0      1
Name: Cluster Labels, dtype: int64

In [78]:
pune_merged = punebangalore_merged[punebangalore_merged['Districtname']=='Pune']
bangalore_merged = punebangalore_merged[punebangalore_merged['Districtname']=='Bangalore']

In [105]:
# create map
map_pune_clusters = folium.Map(location=[PUNE_LATITUDE, PUNE_LONGITUDE], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['#9400D3', '#4B0082', '#0000FF', '#00FF00', '#FFFF00', '#FF7F00', '#FF0000']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pune_merged['Latitude'], pune_merged['Longitude'], pune_merged['Neighbourhood'], pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_pune_clusters)
       
map_pune_clusters

In [82]:
# create map
map_bangalore_clusters = folium.Map(location=[BANGALORE_LATITUDE, BANGALORE_LONGITUDE], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['#9400D3', '#4B0082', '#0000FF', '#00FF00', '#FFFF00', '#FF7F00', '#FF0000']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Latitude'], bangalore_merged['Longitude'], bangalore_merged['Neighbourhood'], bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_bangalore_clusters)
       
map_bangalore_clusters

Examining Clusters

Cluster 0

In [83]:
res = punebangalore_merged.loc[punebangalore_merged['Cluster Labels'] == 0, punebangalore_merged.columns[[1] + list(range(5, punebangalore_merged.shape[1]))]]
print("For cluster 0, the distribution of venues is as:\n{}".format(res['1st Most Common Venue'].value_counts()))
res

For cluster 0, the distribution of venues is as:
Café                        13
Indian Restaurant            8
Department Store             6
Hotel                        5
Coffee Shop                  5
Pizza Place                  3
Bus Station                  2
Athletics & Sports           2
Electronics Store            2
Train Station                2
Park                         2
Bakery                       1
Airport Service              1
Snack Place                  1
Flea Market                  1
Breakfast Spot               1
Fast Food Restaurant         1
Food Court                   1
Sporting Goods Shop          1
Brewery                      1
Henan Restaurant             1
ATM                          1
Mediterranean Restaurant     1
Mobile Phone Shop            1
Sandwich Place               1
South Indian Restaurant      1
Indie Movie Theater          1
Insurance Office             1
Name: 1st Most Common Venue, dtype: int64


,pincode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,560047,"Austin Town, Bangalore, KARNATAKA",12.9613,77.6153,0.0,Hotel,Indian Restaurant,Café,Clothing Store,Bakery,Pub,Chinese Restaurant,Coffee Shop,Donut Shop,Bar
13,560036,"Devasandra, Bangalore, KARNATAKA",12.7572,77.6473,0.0,Henan Restaurant,Zoo Exhibit,Farm,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market
17,560066,"EPIP, Bangalore, KARNATAKA",12.9782,77.7246,0.0,Café,Hotel,Indian Restaurant,Restaurant,Fast Food Restaurant,Brewery,Cafeteria,Soccer Field,Shopping Mall,Pizza Place
21,560051,"H.K.P. Road, Bangalore, KARNATAKA",13.0623,77.6613,0.0,Pizza Place,Convenience Store,Vegetarian / Vegan Restaurant,Brewery,Sandwich Place,Stadium,Restaurant,Bakery,Badminton Court,Italian Restaurant
23,560048,"Hoodi, Bangalore, KARNATAKA",12.9919,77.7162,0.0,Brewery,Indian Restaurant,Coffee Shop,Hotel,Bakery,Platform,Movie Theater,Café,Bus Station,Restaurant
24,560043,"Horamavu, Bangalore, KARNATAKA",13.0273,77.6602,0.0,Department Store,Andhra Restaurant,Badminton Court,Kerala Restaurant,Café,Asian Restaurant,Clothing Store,Vegetarian / Vegan Restaurant,Indian Restaurant,Park
27,560064,"Jakkur, Bangalore, KARNATAKA",13.0785,77.6069,0.0,Café,Hot Spring,Airport,Tennis Court,Lake,Farm,Electronics Store,English Restaurant,Event Space,Exhibit
28,560067,"Kadugodi, Bangalore, KARNATAKA",12.9986,77.761,0.0,Hotel,Train Station,South Indian Restaurant,Pizza Place,Fruit & Vegetable Store,Plaza,Indian Restaurant,Sandwich Place,North Indian Restaurant,Dumpling Restaurant
31,560077,"Kothanur, Bangalore, KARNATAKA",13.0614,77.6494,0.0,Mediterranean Restaurant,Stadium,Supermarket,Coffee Shop,Brewery,Café,North Indian Restaurant,Pizza Place,Grocery Store,Sandwich Place
35,560084,"Lingarajapuram, Bangalore, KARNATAKA",13.0118,77.6278,0.0,Indian Restaurant,Fast Food Restaurant,Convenience Store,Café,Burger Joint,Diner,Park,Kerala Restaurant,Shopping Mall,Chinese Restaurant


Cluster 1

In [96]:
res = punebangalore_merged.loc[punebangalore_merged['Cluster Labels'] == 1, punebangalore_merged.columns[[1] + list(range(5, punebangalore_merged.shape[1]))]]
print("For cluster 1, the distribution of venues is as:\n{}".format(res['1st Most Common Venue'].value_counts()))
res

For cluster 1, the distribution of venues is as:
Scenic Lookout    2
Name: 1st Most Common Venue, dtype: int64


,pincode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,560067,"Medimallasandra, Bangalore, KARNATAKA",12.9523,77.8301,1.0,Scenic Lookout,Zoo Exhibit,Farm,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Dumpling Restaurant
236,411025,"Khanapur Haveli, Pune, MAHARASHTRA",18.4007,73.7318,1.0,Scenic Lookout,Zoo Exhibit,Farm,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Dumpling Restaurant


Cluster 2

In [98]:
res = punebangalore_merged.loc[punebangalore_merged['Cluster Labels'] == 2, punebangalore_merged.columns[[1] + list(range(5, punebangalore_merged.shape[1]))]]
print("For cluster 2, the distribution of venues is as:\n{}".format(res['1st Most Common Venue'].value_counts()))
res

For cluster 2, the distribution of venues is as:
Indian Restaurant       111
Café                      6
Department Store          3
Pizza Place               3
Snack Place               2
Karnataka Restaurant      1
Shopping Mall             1
Gas Station               1
Bakery                    1
Light Rail Station        1
Coffee Shop               1
ATM                       1
Airport Service           1
Fast Food Restaurant      1
Resort                    1
Name: 1st Most Common Venue, dtype: int64


,pincode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560007,"Air Force Hospital, Bangalore, KARNATAKA",12.964,77.6275,2.0,Indian Restaurant,Café,Hotel,Asian Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Pizza Place,Pub,Restaurant
1,560092,"Amruthahalli, Bangalore, KARNATAKA",13.0665,77.5966,2.0,Indian Restaurant,Ice Cream Shop,Department Store,Café,Hotel,Fast Food Restaurant,Brewery,Bubble Tea Shop,Building,Scenic Lookout
2,560045,"Arabic College, Bangalore, KARNATAKA",13.03,77.6209,2.0,Pizza Place,Indian Restaurant,Convenience Store,Clothing Store,BBQ Joint,Jewelry Store,Bus Station,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tea Room
5,560043,"Banaswadi, Bangalore, KARNATAKA",13.0142,77.6519,2.0,Indian Restaurant,Bakery,Café,Falafel Restaurant,Coffee Shop,Pizza Place,Lounge,Bistro,Andhra Restaurant,Korean Restaurant
6,560103,"Bellandur, Bangalore, KARNATAKA",12.9358,77.6668,2.0,Indian Restaurant,Fast Food Restaurant,Café,Kerala Restaurant,Gym,Lounge,Department Store,Dessert Shop,Japanese Restaurant,Multiplex
7,560046,"Benson Town, Bangalore, KARNATAKA",12.9978,77.6042,2.0,Indian Restaurant,Bakery,Middle Eastern Restaurant,Ice Cream Shop,Tea Room,BBQ Joint,Café,Chinese Restaurant,Pizza Place,Fast Food Restaurant
8,560049,"Bhattarahalli, Bangalore, KARNATAKA",13.0258,77.7143,2.0,Indian Restaurant,Café,Hotel,Korean Restaurant,Pizza Place,Bar,Vegetarian / Vegan Restaurant,Falafel Restaurant,Electronics Store,English Restaurant
9,560092,"Byatarayanapura, Bangalore, KARNATAKA",13.0621,77.5964,2.0,Indian Restaurant,Ice Cream Shop,Pizza Place,Café,Fast Food Restaurant,Department Store,Cosmetics Shop,Italian Restaurant,Flea Market,Building
10,560093,"C.V.Raman Nagar, Bangalore, KARNATAKA",12.9856,77.665,2.0,Indian Restaurant,Department Store,Coffee Shop,Café,Fast Food Restaurant,Sandwich Place,Restaurant,Clothing Store,Pizza Place,Mughlai Restaurant
11,560025,"CMP Centre And School, Bangalore, KARNATAKA",12.9583,77.6091,2.0,Indian Restaurant,Hotel,Café,Bakery,Pub,Chinese Restaurant,Coffee Shop,Middle Eastern Restaurant,Donut Shop,Brewery


Cluster 3

In [99]:
res = punebangalore_merged.loc[punebangalore_merged['Cluster Labels'] == 3, punebangalore_merged.columns[[1] + list(range(5, punebangalore_merged.shape[1]))]]
print("For cluster 3 the distribution of venues is as:\n{}".format(res['1st Most Common Venue'].value_counts()))
res

For cluster 3 the distribution of venues is as:
ATM    10
Name: 1st Most Common Venue, dtype: int64


,pincode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,560108,"Anjanapura, Bangalore, KARNATAKA",12.8581,77.5581,3.0,ATM,Asian Restaurant,Pharmacy,Pool,Campground,Photography Lab,Airport Service,Dumpling Restaurant,Flea Market,Fish Market
116,560091,"Herohalli, Bangalore, KARNATAKA",12.9912,77.487,3.0,ATM,Market,Gas Station,Fried Chicken Joint,Electronics Store,Performing Arts Venue,Financial or Legal Service,Field,Fish & Chips Shop,Eastern European Restaurant
123,560058,"Laggere, Bangalore, KARNATAKA",13.0109,77.5207,3.0,ATM,Vegetarian / Vegan Restaurant,Pizza Place,Electronics Store,Gym,Pharmacy,Henan Restaurant,Diner,Field,Fast Food Restaurant
124,560023,"Magadi Road, Bangalore, KARNATAKA",12.989,77.4719,3.0,ATM,Auto Workshop,Resort,Gas Station,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant
158,411015,"Dighi Camp, Pune, MAHARASHTRA",18.6265,73.8661,3.0,ATM,Pharmacy,Coffee Shop,Garden,Hotel,Wine Shop,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
198,411041,"Dhayari, Pune, MAHARASHTRA",18.4374,73.819,3.0,ATM,Cheese Shop,Bus Station,Tea Room,Szechuan Restaurant,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
213,411041,"Nanded, Pune, MAHARASHTRA",18.4548,73.7849,3.0,ATM,Pizza Place,Market,Indian Restaurant,Farm,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit
224,411023,"Shivane, Pune, MAHARASHTRA",18.467,73.7931,3.0,ATM,Pizza Place,Mobile Phone Shop,Market,Diner,Indian Restaurant,Department Store,Gym,Farm,English Restaurant
231,412109,"Dehu, Pune, MAHARASHTRA",18.68,73.7343,3.0,ATM,Train Station,Mobile Phone Shop,Juice Bar,Coworking Space,Creperie,Fish Market,Convenience Store,Fish & Chips Shop,Financial or Legal Service
244,412105,"Moshi, Pune, MAHARASHTRA",18.6523,73.8442,3.0,ATM,Pizza Place,Indian Restaurant,Restaurant,Farm,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant


Cluster 4

In [100]:
res = punebangalore_merged.loc[punebangalore_merged['Cluster Labels'] == 4, punebangalore_merged.columns[[1] + list(range(5, punebangalore_merged.shape[1]))]]
print("For cluster 4 the distribution of venues is as:\n{}".format(res['1st Most Common Venue'].value_counts()))
res

For cluster 4 the distribution of venues is as:
Resort    4
Farm      2
Name: 1st Most Common Venue, dtype: int64


,pincode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,560087,"Gunjur, Bangalore, KARNATAKA",13.4279,77.5131,4.0,Resort,Trail,Lake,Zoo Exhibit,Exhibit,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Falafel Restaurant
46,560064,"Rajanakunte, Bangalore, KARNATAKA",13.1692,77.5639,4.0,Resort,Train Station,Fast Food Restaurant,Zoo Exhibit,Falafel Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit
105,560074,"Ramohalli, Bangalore, KARNATAKA",12.907,77.4185,4.0,Farm,Zoo Exhibit,Farmers Market,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant,Dumpling Restaurant
139,560090,"Tarabanahalli, Bangalore, KARNATAKA",13.1906,77.6279,4.0,Resort,ATM,Diner,Indian Restaurant,Farm,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant
232,411025,"Donje, Pune, MAHARASHTRA",18.4122,73.7666,4.0,Farm,Resort,Hotel,Bed & Breakfast,Indian Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant
248,412115,"Sangrun, Pune, MAHARASHTRA",18.3996,73.6768,4.0,Resort,Zoo Exhibit,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm


Cluster 5

In [101]:
res = punebangalore_merged.loc[punebangalore_merged['Cluster Labels'] == 5, punebangalore_merged.columns[[1] + list(range(5, punebangalore_merged.shape[1]))]]
print("For cluster 5 the distribution of venues is as:\n{}".format(res['1st Most Common Venue'].value_counts()))
res

For cluster 5 the distribution of venues is as:
Indian Restaurant           15
Mobile Phone Shop            2
Maharashtrian Restaurant     1
Convenience Store            1
Hotel                        1
Name: 1st Most Common Venue, dtype: int64


,pincode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,560064,"Attur, Bangalore, KARNATAKA",13.0995,77.568,5.0,Indian Restaurant,Bakery,BBQ Joint,Soccer Field,Bus Station,Asian Restaurant,Andhra Restaurant,Snack Place,Deli / Bodega,Dance Studio
12,560064,"CRPF Campus Yelahanka, Bangalore, KARNATAKA",13.1214,77.5709,5.0,Indian Restaurant,Gas Station,Resort,Coffee Shop,Food Truck,Liquor Store,Dhaba,Exhibit,Electronics Store,English Restaurant
38,560037,"Marathahalli Colony, Bangalore, KARNATAKA",12.9559,77.6882,5.0,Indian Restaurant,Fast Food Restaurant,Airport Terminal,Andhra Restaurant,Clothing Store,Hotel,Dessert Shop,Convenience Store,Restaurant,Lounge
57,560047,"Viveknagar S.O, Bangalore, KARNATAKA",12.9543,77.6227,5.0,Indian Restaurant,Café,Gym / Fitness Center,General Entertainment,Nightclub,Juice Bar,Golf Course,Italian Restaurant,Chinese Restaurant,Bistro
77,560053,"Chickpet, Bangalore, KARNATAKA",12.968,77.5786,5.0,Indian Restaurant,Hotel,Theater,Diner,Bed & Breakfast,Café,Bookstore,Shopping Mall,Breakfast Spot,Seafood Restaurant
82,560100,"Electronics City, Bangalore, KARNATAKA",12.8469,77.6769,5.0,Indian Restaurant,Food Court,Café,Vegetarian / Vegan Restaurant,Bakery,Residential Building (Apartment / Condo),Basketball Court,Bus Stop,Diner,Pool
84,560083,"Gottigere, Bangalore, KARNATAKA",12.8565,77.5877,5.0,Indian Restaurant,Department Store,Food Truck,Italian Restaurant,Gym / Fitness Center,Farmers Market,English Restaurant,Event Space,Exhibit,Falafel Restaurant
108,560061,"Subramanyapura, Bangalore, KARNATAKA",12.8851,77.5416,5.0,Indian Restaurant,South Indian Restaurant,Bus Station,Garden,Arts & Entertainment,Supermarket,Department Store,Theater,Fast Food Restaurant,Farmers Market
110,560109,"Thalaghattapura, Bangalore, KARNATAKA",12.8698,77.5357,5.0,Indian Restaurant,Lounge,Gas Station,Supermarket,Zoo Exhibit,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant
149,411026,"Bhosari I.E., Pune, MAHARASHTRA",18.621,73.8501,5.0,Indian Restaurant,Flea Market,Fast Food Restaurant,Zoo Exhibit,Falafel Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit


Cluster 6

In [106]:
res = punebangalore_merged.loc[punebangalore_merged['Cluster Labels'] == 6, punebangalore_merged.columns[[1] + list(range(5, punebangalore_merged.shape[1]))]]
print("For cluster 6 the distribution of venues is as:\n{}".format(res['1st Most Common Venue'].value_counts()))
res

For cluster 6 the distribution of venues is as:
Reservoir    1
Name: 1st Most Common Venue, dtype: int64


,pincode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
239,412110,"Kolwadi, Pune, MAHARASHTRA",18.8187,73.5038,6.0,Reservoir,Zoo Exhibit,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
